In [5]:
import pandas as pd
from sklearn import model_selection
from sklearn import tree, linear_model, svm
from matplotlib import pyplot as plt

%matplotlib inline

In [33]:
DATA_FILE = "data_table_percentile.tsv"
df = pd.read_csv(DATA_FILE, sep="\t")
label = df["sensitivity_label"].copy()
data = df.drop("sensitivity_label", axis=1)
data.set_index(data.cell_drug_pair, inplace=True)
data.drop("cell_drug_pair", axis=1, inplace=True)

In [41]:
Classifiers = {"Decision tree": tree.DecisionTreeRegressor(), 
               "SVM": svm.SVR(kernel="rbf")}

In [43]:
kf = model_selection.KFold(n_splits=5)
result_df = pd.DataFrame()
fold_num = 0
for train, test in kf.split(df):
    fold_num += 1
    tr, val = model_selection.train_test_split(train, test_size=0.2, random_state=0)
    
    x_train = data.iloc[tr]
    y_train = label.iloc[tr].as_matrix()
    x_val = data.iloc[val]
    y_val = label.iloc[val].as_matrix()
    x_test = data.iloc[test]
    y_test = label.iloc[test].as_matrix()
        
    for classfier_name, clf in Classifiers.iteritems():
        clf.fit(x_train, y_train)
        prediction = clf.predict(x_val)
        mse = metrics.mean_squared_error(prediction, y_val)
        result_df.loc[fold_num, classfier_name] = mse
        
print result_df


        SVM  Decision tree
1  0.031554       0.043081
2  0.029186       0.037847
3  0.027453       0.044009
4  0.027944       0.044998
5  0.025362       0.046323
